In [6]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import os
os.environ['R_HOME'] = 'D:/Program Files/R-4.5.0' #@TODO include in some setup file

from simulation_engine.scenarios.iv.binary_iv import BinaryIV
from simulation_engine.scenarios.iv.continuous_iv import ContinuousIV
from simulation_engine.scenarios.iv.base_iv import IVScenario
from simulation_engine.util.plotting_util import PlottingUtil
import pandas as pd
pd.set_option('display.max_columns', None)


#disable warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
data = BinaryIV.generate_data_rolling_ate(N_simulations=50)

# importr inside loop
#N = 10 -> 17s
#N = 50 -> 1m22s (outside: 18s)

In [ ]:
binaryIV = BinaryIV('IV Dag', data)

binaryIV.bound_ate_2SLS()
binaryIV.bound_ate_causaloptim()
binaryIV.bound_ate_autobound()

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In boundsfunc(p00_0 = p00_0, p00_1 = p00_1, p10_0 = p10_0, p10_1 = p10_1,  :
R[write to console]: 
 
R[write to console]:  Invalid bounds! Data probably does not satisfy the assumptions in the DAG!



ERROR: evaluating object as numeric value: objvar
        (object: <class 'pyomo.core.base.var.ScalarVar'>)
    No value for uninitialized NumericValue object objvar


In [ ]:
binaryIV.data

PlottingUtil.print_bound_statistics(binaryIV.data, ['autobound', 'causaloptim', '2SLS'], 5)W
PlottingUtil.plot_smoothed_ate_vs_bounds(binaryIV.data, ['autobound', 'causaloptim', '2SLS'], 1)


TypeError: PlottingUtil.print_bound_statistics() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
invalids = binaryIV.data[binaryIV.data['causaloptim_bound_valid'] == False]
invalids.mean(numeric_only=True)